In [11]:
using GLPK
using LinearAlgebra
using MathOptInterface
const MOI = MathOptInterface

MathOptInterface

## Loading Data

In [3]:
totaltime, totallines = open("Q1/T1/R1T1.txt") do f
    global lines = readlines(f);
end

450-element Vector{String}:
 "matrix,row,col"
 "S,72,95"
 "I,J,V"
 "8.0,1.0,-1.0"
 "10.0,1.0,1.0"
 "21.0,1.0,-1.0"
 "43.0,1.0,1.0"
 "50.0,1.0,-1.0"
 "51.0,1.0,1.0"
 "8.0,2.0,1.0"
 "9.0,2.0,-1.0"
 "6.0,3.0,-1.0"
 "12.0,3.0,1.0"
 ⋮
 "68.0,-0.5196655879984552"
 "75.0,-1.679092922126729"
 "76.0,-9.871766517721002e-18"
 "81.0,-0.3156306692199074"
 "82.0,-1.1541689385811602"
 "83.0,-0.9060345343182166"
 "86.0,-0.8810837748807233"
 "89.0,-1.2796727037563316"
 "91.0,-1.5079118796617184"
 "92.0,0.05468668371062502"
 "94.0,-0.6437641587296247"
 "95.0,-0.8732197170766789"

In [4]:
function is_info_line(a_line)
    a_line_info = split(a_line, ",");
    line_indicator = a_line_info[1];
    if cmp(line_indicator, "matrix") == 0 || cmp(line_indicator, "vector") == 0
        return true
    end
    return false
end

is_info_line (generic function with 1 method)

In [5]:
function get_info_end_line(lines_info, t_start, t_end_file)
    current_line = lines_info[t_start];
    t = t_start;
    for t in t_start:t_end_file
        current_line = lines_info[t];
        if is_info_line(current_line)
            return t-1
        end
    end
    return t_end_file
end

get_info_end_line (generic function with 1 method)

In [6]:
function fill_matrix(A, n_rows, n_cols, values_vector)
    for info_line in values_vector
        line_info = split(info_line, ",");
        i_v = floor(Int64, parse(Float64, line_info[1]));
        j_v = floor(Int64, parse(Float64, line_info[2]));
        val = parse(Float64, line_info[3]);
        A[i_v, j_v] = val;
    end
end

fill_matrix (generic function with 1 method)

In [7]:
function fill_vector(vec, n_elements, values_vector)
    for info_line in values_vector
        line_info = split(info_line, ",");
        i_v = floor(Int64, parse(Float64, line_info[1]));
        val = parse(Float64, line_info[2]);
        vec[i_v] = val;
    end
end

fill_vector (generic function with 1 method)

In [8]:
function read_block(t_start)
    info_line = lines[t_start];
    info = split(info_line, ",");
    t = t_start + 1;
    
    if cmp(info[1],"matrix") == 0
        matrix_info_line = lines[t];
        matrix_info = split(matrix_info_line, ",");
        matrix_name = matrix_info[1];
        num_rows = parse(Int64, matrix_info[2]);
        num_cols = parse(Int64, matrix_info[3]);
        if cmp(matrix_name, "S") == 0
            global m = num_rows;
            global n = num_cols;
            global S = zeros(m, n);
        else
            @show "Something is not right here! - id:1"
        end
        t = t+1;
        IJV_line = lines[t];
        if cmp(IJV_line, "I,J,V") != 0
            @show "Something is not right here! - id:2"
        end
        t = t+1;
        t_last = get_info_end_line(lines, t, file_num_lines);
        fill_matrix(S, m, n, lines[t:t_last]);
        t = t_last+1;
        
    elseif cmp(info[1],"vector") == 0
        vector_info_line = lines[t];
        vector_info = split(vector_info_line, ",");
        vector_name = vector_info[1];
        num_elements = parse(Int64, vector_info[2]);
        if parse(Int64, vector_info[3]) != 1
            @show "Something is not right here! - id:3"
        end
        l_u_index = 0;
        if cmp(vector_name, "l1") == 0
            global l1 = zeros(num_elements);
            l_u_index = 1;
        elseif cmp(vector_name, "u1") == 0
            global u1 = zeros(num_elements);
            l_u_index = 2;
        else
            @show "Something is not right here! - id:4"
        end
        t = t+1;
        IV_line = lines[t];
        if cmp(IV_line, "I,V") != 0
            @show "Something is not right here! - id:5"
        end
        t = t+1;
        t_last = get_info_end_line(lines, t, file_num_lines);
        if l_u_index == 1
            fill_vector(l1, num_elements, lines[t:t_last]);
        else
            fill_vector(u1, num_elements, lines[t:t_last]);
        end
        t = t_last+1;
    end
    return t
end

read_block (generic function with 1 method)

In [9]:
file_num_lines = length(lines)
m = 0;
n = 0;
line_counter = 1;
while line_counter < file_num_lines
    line_counter = read_block(line_counter);
end

## Round 1 Question:

$$
find  \qquad\qquad v \\
s.t.: \quad Sv = 0 \\
 \qquad\qquad l^1 \leq v \leq u^1
$$

### Solving: Simple Objective Function f(v)=0

In [12]:
optimizer = GLPK.Optimizer()
v = MOI.add_variables(optimizer, n)
# ------------------------------------------------------------------------------------
objective_function = MOI.ScalarAffineFunction(MOI.ScalarAffineTerm.(zeros(n), v), 0.0)
MOI.set(optimizer, 
        MOI.ObjectiveFunction{MOI.ScalarAffineFunction{Float64}}(),
        objective_function)
MOI.set(optimizer, MOI.ObjectiveSense(), MOI.MIN_SENSE)
# ------------------------------------------------------------------------------------
for i in 1:n
        MOI.add_constraint(optimizer, 
                           MOI.SingleVariable(v[i]), 
                           MOI.Interval(l1[i], u1[i]))
end
# ------------------------------------------------------------------------------------
for (i,row) in enumerate(eachrow(S))
        row_function = MOI.ScalarAffineFunction(MOI.ScalarAffineTerm.(row, v), 0.0);
        MOI.add_constraint(optimizer, row_function, MOI.EqualTo(0.0))
end

In [14]:
MOI.optimize!(optimizer)
status = MOI.get(optimizer, MOI.TerminationStatus())
if status != MOI.OPTIMAL
    @show "Unsuccessful to reach optimal value"
end
v0 = MOI.get(optimizer, MOI.VariablePrimal(), v);

## Saving

In [15]:
f = open("Q1/T1/output.txt", "w") do f
  for data_line in v0
    println(f, data_line)
  end
end